# Matrix Encryption
Makenna Worley and Nic Van Der Werf

In [15]:
# Importing and declaring global variables
import numpy as np
import math
import random

MODULO = 59
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!?&\n"
print(len(ALPHABET))

KEY_MATRIX_2x2 = np.array([
    [3, 2],
    [5, 7]
])

text1 = open('text/one.txt').read()
text2 = open('text/two.txt').read()
text3 = open('text/three.txt').read()
text4 = open('text/four.txt').read()

59


In [16]:
def generate_key_matrix(n, modulo):
    while True:
        key_matrix = np.random.randint(0, modulo, size=(n, n))
        # Compute determinant mod
        a, b = key_matrix[0]
        c, d = key_matrix[1]
        det = (a*d - b*c) % modulo
        if math.gcd(det, modulo) == 1:
            try:
                inverse_key_matrix = inverse_2x2_mod(key_matrix, modulo)
                return key_matrix, inverse_key_matrix
            except ValueError:
                continue
        else:
            print("Failed Iteration")


def generate_key_matrix_nxn(n, modulo):
    while True:
        key_matrix = np.random.randint(0, modulo, size=(n, n))
        # Compute determinant mod
        det = int(round(np.linalg.det(key_matrix))) % modulo
        if math.gcd(det, modulo) == 1:
            try:
                inverse_key_matrix = inverse_nxn_mod(key_matrix, modulo)
                return key_matrix, inverse_key_matrix
            except ValueError:
                continue
        else:
            # Not invertible, try again
            pass


def inverse_2x2_mod(matrix, modulo):
    # Compute modular inverse for a 2x2 matrix using integer arithmetic only
    a, b = matrix[0]
    c, d = matrix[1]
    det = int((a*d - b*c) % modulo)
    if math.gcd(det, modulo) != 1:
        raise ValueError("Matrix not invertible modulo {}".format(modulo))
    det_inv = pow(det, -1, modulo)
    # Adjugate matrix
    return (det_inv * np.array([[d, -b],
                                [-c, a]])) % modulo


def inverse_nxn_mod(matrix, modulo):
    # Convert to integers and work on a copy
    A = np.array(matrix, dtype=int) % modulo
    n = A.shape[0]

    # Create identity matrix
    I = np.eye(n, dtype=int)

    # Perform Gaussian elimination
    for i in range(n):
        # Find pivot (row with non-zero element in column i)
        for r in range(i, n):
            if A[r, i] % modulo != 0:
                # Swap pivot row if necessary
                if r != i:
                    A[[i, r]] = A[[r, i]]
                    I[[i, r]] = I[[r, i]]
                break
        else:
            # No pivot found, matrix not invertible
            raise ValueError("Matrix not invertible modulo {}".format(modulo))

        # Normalize pivot row
        inv_pivot = pow(int(A[i, i]), -1, modulo)
        A[i] = (A[i] * inv_pivot) % modulo
        I[i] = (I[i] * inv_pivot) % modulo

        # Eliminate other rows
        for r in range(n):
            if r != i:
                factor = A[r, i] % modulo
                if factor != 0:
                    A[r] = (A[r] - factor * A[i]) % modulo
                    I[r] = (I[r] - factor * I[i]) % modulo

    return I % modulo


def encrypt(plaintext, key_matrix, modulo, alphabet):
    # Map chars to numbers from 0 to len(alphabet)-1
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)  # Use a padding value outside the char range
    # Convert plaintext to numbers
    numbers = [char_to_num[c] for c in plaintext if c in char_to_num]
    # Pad if necessary
    if len(numbers) % 2 != 0:
        numbers.append(padding_val)

    plaintext_matrix = np.array(numbers).reshape(-1, 2).T
    ciphertext_matrix = (key_matrix.dot(plaintext_matrix) % modulo).T
    # Convert back to chars (skip padding_val when converting back)
    ciphertext = []
    for pair in ciphertext_matrix:
        for num in pair:
            if num != padding_val:
                if num in num_to_char:
                    ciphertext.append(num_to_char[num])
                else:
                    # If something unexpected, place a placeholder
                    ciphertext.append('?')
    return ''.join(ciphertext), ciphertext_matrix


def encrypt_nxn(plaintext, key_matrix, modulo, alphabet):
    # Map chars to numbers from 0 to len(alphabet)-1
    n = key_matrix.shape[0]
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)  # Use a padding value outside the char range
    # Convert plaintext to numbers
    numbers = [char_to_num[c] for c in plaintext if c in char_to_num]
    # Pad if necessary
    remainder = len(numbers) % n
    if remainder != 0:
        numbers.extend([padding_val] * (n - remainder))

    plaintext_matrix = np.array(numbers).reshape(-1, n).T
    ciphertext_matrix = (key_matrix.dot(plaintext_matrix) % modulo).T
    # Convert back to chars (skip padding_val when converting back)
    ciphertext = []
    for pair in ciphertext_matrix:
        for num in pair:
            if num != padding_val:
                if num in num_to_char:
                    ciphertext.append(num_to_char[num])
                else:
                    # If something unexpected, place a placeholder
                    ciphertext.append('?')
    return ''.join(ciphertext), ciphertext_matrix


def decrypt(ciphertext, inverse_key_matrix, modulo, alphabet):
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)

    numbers = [char_to_num[c] for c in ciphertext if c in char_to_num]
    if len(numbers) % 2 != 0:
        numbers.append(padding_val)

    cipher_matrix = np.array(numbers).reshape(-1, 2)
    decrypted_matrix = (inverse_key_matrix.dot(cipher_matrix.T) % modulo).T

    # Convert back to text and remove padding
    plaintext_chars = []
    for pair in decrypted_matrix:
        for num in pair:
            if num == padding_val:
                # Padding encountered, skip it
                continue
            if num in num_to_char:
                plaintext_chars.append(num_to_char[num])
            else:
                plaintext_chars.append('?')
    return ''.join(plaintext_chars)


def decrypt_nxn(ciphertext, inverse_key_matrix, modulo, alphabet):
    n = inverse_key_matrix.shape[0]
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)

    numbers = [char_to_num[c] for c in ciphertext if c in char_to_num]
    remainder = len(numbers) % n
    if remainder != 0:
        numbers.extend([padding_val] * (n - remainder))

    cipher_matrix = np.array(numbers).reshape(-1, n)
    decrypted_matrix = (inverse_key_matrix.dot(cipher_matrix.T) % modulo).T

    # Convert back to text and remove padding
    plaintext_chars = []
    for pair in decrypted_matrix:
        for num in pair:
            if num == padding_val:
                # Padding encountered, skip it
                continue
            if num in num_to_char:
                plaintext_chars.append(num_to_char[num])
            else:
                plaintext_chars.append('?')
    dec = ''.join(plaintext_chars)
    return dec.rstrip('A')


def generate_random_string(alphabet, l):
    return ''.join(random.choice(alphabet) for _ in range(l))


def test_encryption_decryption(l):
    plaintext = generate_random_string(ALPHABET, l)
    ciphertext, ciphertext_matrix = encrypt(plaintext, KEY_MATRIX_2x2, MODULO, ALPHABET)
    decrypted_text = decrypt(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
    return plaintext, decrypted_text, plaintext == decrypted_text

In [17]:
inverse_key_matrix = inverse_2x2_mod(KEY_MATRIX_2x2, MODULO)

for idx in range(1, 51):
    plaintext, decrypted_text, result = test_encryption_decryption(200)
    if result:
        print(f"Test {idx}: True")
    else:
        print(f"Test {idx}: False")
        print(f"  Original: {plaintext}")
        print(f"  Decrypted: {decrypted_text}")

Test 1: True
Test 2: True
Test 3: True
Test 4: True
Test 5: True
Test 6: True
Test 7: True
Test 8: True
Test 9: True
Test 10: True
Test 11: True
Test 12: True
Test 13: True
Test 14: True
Test 15: True
Test 16: True
Test 17: True
Test 18: True
Test 19: True
Test 20: True
Test 21: True
Test 22: True
Test 23: True
Test 24: True
Test 25: True
Test 26: True
Test 27: True
Test 28: True
Test 29: True
Test 30: True
Test 31: True
Test 32: True
Test 33: True
Test 34: True
Test 35: True
Test 36: True
Test 37: True
Test 38: True
Test 39: True
Test 40: True
Test 41: True
Test 42: True
Test 43: True
Test 44: True
Test 45: True
Test 46: True
Test 47: True
Test 48: True
Test 49: True
Test 50: True


In [18]:
inverse_key_matrix = inverse_2x2_mod(KEY_MATRIX_2x2, MODULO)
print(f"Key Matrix:\n{KEY_MATRIX_2x2}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

ciphertext, ciphertext_matrix = encrypt(text2, KEY_MATRIX_2x2, MODULO, ALPHABET)
print(f"Ciphertext Matrix:\n{ciphertext_matrix}")

decrypted_text = decrypt(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print("Original Text:", text2)
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted Text: {decrypted_text}")
print(f"Condition: {np.linalg.cond(KEY_MATRIX_2x2)}")


Key Matrix:
[[3 2]
 [5 7]]
Inverse Key Matrix:
[[ 6 32]
 [21 11]]
Ciphertext Matrix:
[[30 37]
 [56 28]
 [21  1]
 [48 25]
 [30 44]
 [27 37]
 [16  6]
 [35 50]
 [51 34]
 [ 6 30]
 [24 37]
 [ 8 31]
 [18 24]
 [ 2 10]
 [50 36]
 [31 29]
 [44 28]
 [45 10]
 [ 3 30]
 [10 44]
 [41 21]
 [35 43]
 [45 13]
 [43 56]
 [39 22]]
Original Text: Hi Jonas, we would really like an A in the class.

Ciphertext: el?cVBwZesblQGjyziGeYlIfSYCKykfdsctKDeKspVjrtNr?nW
Decrypted Text: Hi Jonas, we would really like an A in the class.

Condition: 7.780565547548909


In [20]:
key_matrix, inverse_key_matrix = generate_key_matrix(2, MODULO)

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

ciphertext, ciphertext_matrix = encrypt(text4, key_matrix, MODULO, ALPHABET)
print(f"Ciphertext Matrix:\n{ciphertext_matrix}")

decrypted_text = decrypt(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print("Original Text:", text4)
print()
print(f"Ciphertext: {ciphertext}")
print()
print(f"Decrypted Text: {decrypted_text}")
print()
print(f"Condition: {np.linalg.cond(key_matrix)}")

Key Matrix:
[[26 47]
 [52 46]]
Inverse Key Matrix:
[[ 8 38]
 [32 43]]
Ciphertext Matrix:
[[35 46]
 [50 17]
 [58 14]
 ...
 [15 12]
 [53 15]
 [28 45]]
Original Text: We're no strangers to love
You know the rules and so do I
A full commitment's what I'm thinkin' of
You wouldn't get this from any other guy
I just wanna tell you how I'm feeling
Gotta make you understand
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
We've known each other for so long
Your heart's been aching, but you're too shy to say it
Inside, we both know what's been going on
We know the game and we're gonna play it
And if you ask me how I'm feeling
Don't tell me you're too blind to see
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Never gonna give you up
Never gonna let you

In [22]:
keyMatrixSize = 7

key_matrix, inverse_key_matrix = generate_key_matrix_nxn(keyMatrixSize, MODULO)

ciphertext, ciphertext_matrix = encrypt_nxn(text1, key_matrix, MODULO, ALPHABET)
print("Plain Text Length:", len(text1))
print("Ciphertext Length:", len(ciphertext))

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

decrypted_text = decrypt_nxn(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print("Original Text:", text1)
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted Text: {decrypted_text}")

Plain Text Length: 21
Ciphertext Length: 21
Key Matrix:
[[20 48 50 39 40 10 22]
 [40 49 23 16 36 28 50]
 [15 34 25 29 17 35 21]
 [42 49 54 31  7 24 51]
 [15 10 41 52 28  9 17]
 [22 20 13 26 55 49 44]
 [40  0  1 38 51 47 18]]
Inverse Key Matrix:
[[ 3 38 45 53 58 39 10]
 [54  3 41 40 48 49 55]
 [43 49 38 17 32 38 12]
 [11 31 15 57 24 45 28]
 [51 20 57  2 57 11 57]
 [41 58  6 18 57 13  9]
 [21 49  7  0  0 33 51]]
Original Text: Hello My name is Nic

Ciphertext: AWichnzrxOvIa,Yicrw
p
Decrypted Text: Hello My name is Nic

